In [1]:
# """PDB dataset loader."""
# import math
# from typing import Optional

# import torch
# import torch.distributed as dist

# import tree
# import numpy as np
# import torch
# import pandas as pd
# import logging
# import random
# import functools as fn

# from torch.utils import data
# from data import utils as du
# from openfold.data import data_transforms
# from openfold.np import residue_constants
# from openfold.utils import rigid_utils

# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.linear_model import LinearRegression

In [2]:
from openfold.np import residue_constants

In [3]:
import pandas as pd

In [4]:

import torch
import tree
#clear memory better
with torch.no_grad():
    torch.cuda.empty_cache()
import numpy as np
import util.npose_util as nu
import os
import pathlib
import dgl
from dgl import backend as F
import torch_geometric
from torch.utils.data import random_split, DataLoader, Dataset
from typing import Dict
from torch import Tensor
from dgl import DGLGraph
from torch import nn
# from chemical import cos_ideal_NCAC #from RoseTTAFold2
from torch import einsum
import time
import random
torch.cuda.is_available()

True

In [5]:
import se3_diffuse.utils as du
from data_rigid_diffuser.diffuser import FrameDiffNoise
from gudiff_model import Data_Graph
from gudiff_model.Data_Graph import build_npose_from_coords, dump_coord_pdb, define_graph_edges, make_pe_encoding
from gudiff_model.Data_Graph import Helix4_Dataset, Make_KNN_MP_Graphs
from se3_transformer.model.basis import get_basis, update_basis_with_fused
from se3_transformer.model.transformer import Sequential, SE3Transformer
from se3_transformer.model.transformer_topk import SE3Transformer_topK
from se3_transformer.model.FAPE_Loss import FAPE_loss, Qs2Rs, normQ
from se3_transformer.model.layers.attentiontopK import AttentionBlockSE3
from se3_transformer.model.layers.linear import LinearSE3
from se3_transformer.model.layers.convolution import ConvSE3, ConvSE3FuseLevel
from se3_transformer.model.layers.norm import NormSE3
from se3_transformer.model.layers.pooling import GPooling, Latent_Unpool, Unpool_Layer
from se3_transformer.runtime.utils import str2bool, to_cuda
from se3_transformer.model.fiber import Fiber
from se3_transformer.model.transformer import get_populated_edge_features

In [6]:
#indices for, unsure if needed
CA = Data_Graph.CA
N = Data_Graph.N
C = Data_Graph.C

#find better way to incorporate coord_scale

#needed
N_CA_dist = (Data_Graph.N_CA_dist/10.).to('cuda')
C_CA_dist = (Data_Graph.C_CA_dist/10.).to('cuda')

In [7]:
#basically a I just need data loader to grab coordinates from datasets

In [8]:
#########CURRENTLY REWRITING THIS######

def torch_normalize(v, eps=1e-6):
    """Normalize vector in last axis"""
    norm = torch.linalg.vector_norm(v, dim=len(v.shape)-1)+eps
    return v / norm[...,None]
def gudiff_parse_chain_feats(chain_feats, scale_factor=10., cast_type=torch.float32):
    ca_idx = residue_constants.atom_order['CA']
    n_idx = residue_constants.atom_order['N']
    c_idx = residue_constants.atom_order['C']
    chain_feats['bb_mask'] = chain_feats['atom_mask'][:, ca_idx]
    
    bb_pos = chain_feats['atom_positions'][:, ca_idx]/scale_factor #scale factor mod
    bb_center = np.sum(bb_pos, axis=0) / (np.sum(chain_feats['bb_mask']) + 1e-5)
    centered_pos = chain_feats['atom_positions'] - bb_center[None, None, :]
    
    
    coordinates = centered_pos/scale_factor
    #unsqueeze to stack together later
    N_CA_vec = torch.tensor(coordinates[:,n_idx] - coordinates[:,ca_idx], dtype=cast_type)/scale_factor
    C_CA_vec = torch.tensor(coordinates[:,c_idx] - coordinates[:,ca_idx], dtype=cast_type)/scale_factor
        
    N_CA_vec = torch_normalize(N_CA_vec)#.unsqueeze(2) #do the unsqueeze later
    C_CA_vec = torch_normalize(C_CA_vec)#.unsqueeze(2)

    scaled_pos = centered_pos / scale_factor
    chain_feats['atom_positions'] = scaled_pos * chain_feats['atom_mask'][..., None]
    
    chain_feats['CA'] = torch.tensor(coordinates[:,ca_idx],dtype=cast_type)
    chain_feats['N_CA_vec'] = N_CA_vec
    chain_feats['C_CA_vec'] = C_CA_vec
    return chain_feats



class smallPDBDataset(torch.utils.data.Dataset):
    def __init__(
            self,
            diffuser,
            meta_data_path = '/mnt/h/datasets/p200/metadata.csv',
            filter_dict=True,
            maxlen=None,
            is_training=True,
            input_t=None
        ):
        #self._log = logging.getLogger(__name__)
        self._is_training = is_training
        self.meta_data_path = meta_data_path
        self._init_metadata(filter_dict=filter_dict,maxlen=maxlen) #includes create split that saves self.csv
        self._diffuser = diffuser
        self.input_t = input_t
        
    @property
    def is_training(self):
        return self._is_training

    @property
    def diffuser(self):
        return self._diffuser

    @property
    def data_conf(self):
        return self._data_conf

    def _init_metadata(self, filter_dict=True, maxlen=None):
        """Initialize metadata."""
        
        #meta_data_path = '/mnt/h/datasets/p200/metadata.csv'
        pdb_csv = pd.read_csv(self.meta_data_path)
        
        if filter_dict:
            filter_conf = {'allowed_oligomer': ['monomeric'],
                           'max_loop_percent': 0.75}
            pdb_csv = pdb_csv[pdb_csv.oligomeric_detail.isin(filter_conf['allowed_oligomer'])]
            pdb_csv = pdb_csv[pdb_csv.coil_percent < filter_conf['max_loop_percent']]
            pdb_csv = pdb_csv.sort_values('modeled_seq_len', ascending=False)
            
        if maxlen is not None:
            pdb_csv = pdb_csv[:maxlen]
        #self._create_split(pdb_csv)
        self.csv = pdb_csv
    def _create_split(self, pdb_csv):
        # Training or validation specific logic.
        #if self.is_training:
        self.csv = pdb_csv
        #self._log.info(
        #    f'Training: {len(self.csv)} examples')
#         else:
#             all_lengths = np.sort(pdb_csv.modeled_seq_len.unique())
#             length_indices = (len(all_lengths) - 1) * np.linspace(
#                 0.0, 1.0, self._data_conf.num_eval_lengths)
#             length_indices = length_indices.astype(int)
            
#             if self._simple:
#                 eval_lengths = np.array([65]).astype(int)
#             else:
#                 eval_lengths = all_lengths[length_indices]
                
#             eval_csv = pdb_csv[pdb_csv.modeled_seq_len.isin(eval_lengths)]
#             # Fix a random seed to get the same split each time.
#             eval_csv = eval_csv.groupby('modeled_seq_len').sample(
#                 self._data_conf.samples_per_eval_length, replace=True, random_state=123)
#             eval_csv = eval_csv.sort_values('modeled_seq_len', ascending=False)
#             self.csv = eval_csv
#             self._log.info(
#                 f'Validation: {len(self.csv)} examples with lengths {eval_lengths}')
    # cache make the same sample in same batch 
    #@fn.lru_cache(maxsize=100)
    def _process_csv_row(self, processed_file_path, index=0):
        
        processed_feats = du.read_pkl(processed_file_path)
        chain_feats = gudiff_parse_chain_feats(processed_feats,scale_factor=10.)
        
        # Only take modeled residues.
        modeled_idx = processed_feats['modeled_idx']
        min_idx = np.min(modeled_idx)
        max_idx = np.max(modeled_idx)
        del processed_feats['modeled_idx']
        processed_feats = tree.map_structure(
            lambda x: x[min_idx:(max_idx+1)], processed_feats)
        chain_feats = tree.map_structure(
            lambda x: x[min_idx:(max_idx+1)], chain_feats)
        

        # Run through OpenFold data transforms.
        # Re-number residue indices for each chain such that it starts from 1.
        # Randomize chain indices.
        chain_idx = processed_feats["chain_index"]
        res_idx = processed_feats['residue_index']
        new_res_idx = np.zeros_like(res_idx)
        new_chain_idx = np.zeros_like(res_idx)
        all_chain_idx = np.unique(chain_idx).tolist()
        shuffled_chain_idx = np.array(
            random.sample(all_chain_idx, len(all_chain_idx))) - np.min(all_chain_idx) + 1
        for i,chain_id in enumerate(all_chain_idx):
            chain_mask = (chain_idx == chain_id).astype(int)
            chain_min_idx = np.min(res_idx + (1 - chain_mask) * 1e3).astype(int)
            new_res_idx = new_res_idx + (res_idx - chain_min_idx + 1) * chain_mask

            # Shuffle chain_index
            replacement_chain_id = shuffled_chain_idx[i]
            new_chain_idx = new_chain_idx + replacement_chain_id * chain_mask

        # To speed up processing, only take necessary features
        final_feats = {
            'chain_idx': new_chain_idx,
            'residue_index': processed_feats['residue_index'],
            'res_mask': processed_feats['bb_mask'],
            'CA':   chain_feats['CA'],
            'N_CA': chain_feats['N_CA_vec'], #when unsqueeze? later maybe take time to change this behavior
            'C_CA': chain_feats['C_CA_vec'],
            'file_path_index' : index
        }
        return final_feats
    
    def get_specific_t(self, idx, t):
        example_idx = idx
        csv_row = self.csv.iloc[example_idx]
        if 'pdb_name' in csv_row:
            pdb_name = csv_row['pdb_name']
        elif 'chain_name' in csv_row:
            pdb_name = csv_row['chain_name']
        else:
            raise ValueError('Need chain identifier.')
            
        processed_file_path = csv_row['processed_path']
        chain_feats = self._process_csv_row(processed_file_path,index=idx)

        bb_noised =  self._diffuser.forward(chain_feats, t=t)
        chain_feats.update(bb_noised)

        # Convert all features to tensors.
        final_feats = tree.map_structure(
                    lambda x: x if torch.is_tensor(x) else torch.tensor(x), chain_feats)
        
        return final_feats
        
        
    def __getitem__(self, idx):
        example_idx = idx
        csv_row = self.csv.iloc[example_idx]
        if 'pdb_name' in csv_row:
            pdb_name = csv_row['pdb_name']
        elif 'chain_name' in csv_row:
            pdb_name = csv_row['chain_name']
        else:
            raise ValueError('Need chain identifier.')
        processed_file_path = csv_row['processed_path']
        chain_feats = self._process_csv_row(processed_file_path, index=idx)

        # Use a fixed seed for evaluation.
#         if self.is_training:
        rng = np.random.default_rng(None)
#         else:
#             rng = np.random.default_rng(idx)

        # Sample t and diffuse.
#         if self.is_training:
        if self.input_t is None:
            t = rng.uniform(1e-3, 1.0)
        else:
            t = self.input_t
        bb_noised =  self._diffuser.forward(chain_feats, t=t)
#         else:
#             t = 1.0
#             diff_feats_t = self.diffuser.sample_ref(
#                 n_samples=gt_bb_rigid.shape[0],
#                 impute=gt_bb_rigid,
#                 diffuse_mask=None,
#                 as_tensor_7=True,
#             )
        chain_feats.update(bb_noised)

        # Convert all features to tensors.
        final_feats = tree.map_structure(
            lambda x: x if torch.is_tensor(x) else torch.tensor(x), chain_feats)
        #final_feats = du.pad_feats(final_feats, csv_row['modeled_seq_len'])
        #if self.is_training:
#         else:
#             return final_feats, pdb_name
        
        return final_feats
    
    def __len__(self):
        return len(self.csv)

In [9]:
# from torch.utils import data

class TrainSampler(torch.utils.data.Sampler):

    def __init__(self, batch_size, dataset,
                 sample_mode='length_batch'):
        
        self.dataset = dataset
        self._data_csv = dataset.csv
        self._dataset_indices = list(range(len(self._data_csv)))
        self._data_csv['index'] = self._dataset_indices
        self._batch_size = batch_size
        self.epoch = 0
        self._sample_mode = sample_mode
        self.sampler_len = len(self._dataset_indices) * self._batch_size
        self.min_t = 1e-3
        #self._log = logging.getLogger(__name__)
        #self._data_conf = data_conf
        #self._dataset = dataset
        #self._data_csv = self._dataset.csv
    def __iter__(self):
        if self._sample_mode == 'length_batch':
            # Each batch contains multiple proteins of the same length.
            sampled_order = self._data_csv.groupby('modeled_seq_len').sample(
                self._batch_size, replace=True, random_state=self.epoch) #one batch per length
            return iter(sampled_order['index'].tolist())
        elif self._sample_mode == 'single_length':
            rand_index = self._data_csv['index'].to_numpy()
            np.random.shuffle(rand_index)
            num_batches = int(rand_index.shape[0]/self._batch_size)
            rand_index = rand_index[:(num_batches*self._batch_size)] #drop last batch
            return iter(rand_index)
        else:
            raise ValueError(f'Invalid sample mode: {self._sample_mode}')
    
#     def getbb(self, idx):
#         csv_row = self._data_csv.iloc[idx]
#         processed_file_path = csv_row['processed_path']
#         chain_feats = self.dataset._process_csv_row(processed_file_path)
#         return chain_feats  
            
    def __len__(self):
        return len(self.csv)

    def set_epoch(self, epoch):
        self.epoch = epoch

    def __len__(self):
        return self.sampler_len

In [10]:
from gudiff_model.Graph_UNet import GraphUNet

In [11]:
def model_step(batch_feats, noised_dict, graph_maker, graph_unet, train=True):
    L = batch_feats['CA'].shape[1]
    B = batch_feats['CA'].shape[0]
    CA_t  = batch_feats['CA']
    NC_t = CA_t +  batch_feats['N_CA']
    CC_t = CA_t +  batch_feats['C_CA']
    true =  torch.cat((NC_t,CA_t,CC_t),dim=2).reshape(B,L,3,3)

    CA_n  = batch_feats['CA_noised'].reshape(B, L, 3)#.to('cuda')
    NC_n = CA_n + batch_feats['N_CA_noised'].reshape(B, L, 3)#.to('cuda')
    CC_n = CA_n + batch_feats['C_CA_noised'].reshape(B, L, 3)#.to('cuda')
    noise_xyz =  torch.cat((NC_n,CA_n,CC_n),dim=2).reshape(B,L,3,3)
    
    x = graph_maker.prep_for_network(noised_dict)
    out = graph_unet(x, batch_feats['t'])
    CA_p = out['1'][:,0,:].reshape(B, L, 3) + CA_n #translation of Calpha
    Qs = out['1'][:,1,:] # rotation of frame
    Qs = Qs.unsqueeze(1).repeat((1,2,1))
    Qs = torch.cat((torch.ones((B*L,2,1),device=Qs.device),Qs),dim=-1).reshape(B,L,2,4)
    Qs = normQ(Qs)
    Rs = Qs2Rs(Qs)
    N_C_to_Rot = torch.cat((noised_dict['N_CA'].reshape(B, L, 3),
                            noised_dict['C_CA'].reshape(B, L, 3)),dim=2).reshape(B,L,2,1,3)
    
    rot_vecs = einsum('bnkij,bnkhj->bnki',Rs, N_C_to_Rot)
    NC_p = CA_p + rot_vecs[:,:,0,:]*N_CA_dist #remove bc who cares? me maybe
    CC_p = CA_p + rot_vecs[:,:,1,:]*C_CA_dist #remove maybe

    pred = torch.cat((NC_p,CA_p,CC_p),dim=2).reshape(B,L,3,3)

    tloss, loss = FAPE_loss(pred.unsqueeze(0), true, batch_feats['score_scale'])
    
    return pred, tloss

def get_noise_pred_true(batch_feats, noised_dict, graph_maker, graph_unet, scale=10):
    
    L = batch_feats['CA'].shape[1]
    B = batch_feats['CA'].shape[0]
    CA_t  = batch_feats['CA']
    NC_t = CA_t +  batch_feats['N_CA']*N_CA_dist
    CC_t = CA_t +  batch_feats['C_CA']*C_CA_dist
    true =  torch.cat((NC_t,CA_t,CC_t),dim=2).reshape(B,L,3,3)

    CA_n  = batch_feats['CA_noised'].reshape(B, L, 3)#.to('cuda')
    NC_n = CA_n + batch_feats['N_CA_noised'].reshape(B, L, 3)*N_CA_dist
    CC_n = CA_n + batch_feats['C_CA_noised'].reshape(B, L, 3)*C_CA_dist
    noise_xyz =  torch.cat((NC_n,CA_n,CC_n),dim=2).reshape(B,L,3,3)
    
    x = graph_maker.prep_for_network(noised_dict)
    out = graph_unet(x, batch_feats['t'])
    CA_p = out['1'][:,0,:].reshape(B, L, 3) + CA_n #translation of Calpha
    Qs = out['1'][:,1,:] # rotation of frame
    Qs = Qs.unsqueeze(1).repeat((1,2,1))
    Qs = torch.cat((torch.ones((B*L,2,1),device=Qs.device),Qs),dim=-1).reshape(B,L,2,4)
    Qs = normQ(Qs)
    Rs = Qs2Rs(Qs)
    N_C_to_Rot = torch.cat((noised_dict['N_CA'].reshape(B, L, 3),
                            noised_dict['C_CA'].reshape(B, L, 3)),dim=2).reshape(B,L,2,1,3)
    
    rot_vecs = einsum('bnkij,bnkhj->bnki',Rs, N_C_to_Rot)
    NC_p = CA_p + rot_vecs[:,:,0,:]*N_CA_dist #remove bc who cares? me maybe
    CC_p = CA_p + rot_vecs[:,:,1,:]*C_CA_dist #remove maybe

    pred = torch.cat((NC_p,CA_p,CC_p),dim=2).reshape(B,L,3,3)
    
    return true.to('cpu').numpy()*scale, noise_xyz.to('cpu').numpy()*scale, pred.detach().to('cpu').numpy()*scale

def generate_tbatch(pdbdataset, index_in, input_t):
    batch_list = []
    for i,t in enumerate(input_t):
        batch_list.append(pdbdataset.get_specific_t(index_in[i], input_t[i]))

    batch_feats = {}
    for k in batch_list[0].keys():
        batch_feats[k] = torch.stack([batch_list[i][k] for i in range(len(batch_list))])
    return batch_feats


def dump_tnp(true, noise, pred, t_val, e=0, numOut=1,outdir='output/'):
    
    if numOut>true.shape[0]:
        numOut = true.shape[0]
    
    for x in range(numOut):
        dump_coord_pdb(true[x], fileOut=f'{outdir}/true_{t_val[x]*100:.0f}_e{e}_{x}.pdb')
        dump_coord_pdb(noise[x], fileOut=f'{outdir}/noise_{t_val[x]*100:.0f}_e{e}_{x}.pdb')
        dump_coord_pdb(pred[x], fileOut=f'{outdir}/pred_{t_val[x]*100:.0f}_e{e}_{x}.pdb')
        
def visualize_model(pdbdataset, view_t, epoch=1, numOut=1, outdir='output/'):
    device='cuda'
    index_in = np.random.choice(np.arange(len(pdbdataset)), size=len(view_t))
    batch_feats = generate_tbatch(pdbdataset , index_in, view_t)
    
    batch_feats= tree.map_structure(
                    lambda x: x.to(device), batch_feats)
    noised_dict =   {'CA': batch_feats['CA_noised'] ,
                     'N_CA': batch_feats['N_CA_noised'].unsqueeze(-2) ,
                     'C_CA': batch_feats['C_CA_noised'].unsqueeze(-2)  }
    
    true, noise, pred = get_noise_pred_true(batch_feats, noised_dict, gm, gu, scale=10)
    dump_tnp(true,noise,pred, view_t, e=epoch, numOut=numOut, outdir=f'{outdir}/models/')

In [12]:
def make_save_folder(name=''):
    base_folder = time.strftime(f'log/%y%b%d_%I%M%p_{name}/', time.localtime())
    if not os.path.exists(base_folder):
        os.makedirs(base_folder)
    subfolders = ['models']
    for subfolder in subfolders:
        if not os.path.exists(base_folder + subfolder):
            os.makedirs(base_folder + subfolder)
            
    return base_folder
        
def save_chkpt(model_path, model, optimizer, epoch, batch, val_losses, train_losses):
    """Save a training checkpoint
    Args:
        model_path (str): the path to save the model to
        model (nn.Module): the model to save
        optimizer (torch.optim.Optimizer): the optimizer to save
        epoch (int): the current epoch
        batch (int): the current batch in the epoch
        loss_domain (list of int): a list of the shared domain for val and training 
            losses
        val_losses (list of float): a list containing the validation losses
        train_losses (list of float): a list containing the training losses
    """
    os.makedirs(os.path.dirname(model_path), exist_ok=True)
    state_dict = dict()
    state_dict.update({'model':model.state_dict(),
                       'optimizer':optimizer.state_dict(),
                       'epoch':epoch,
                       'batch':batch,
                       'train_losses':train_losses,
                       'val_losses':val_losses
                       })
    torch.save(state_dict, f'{model_path}model_e{epoch}')
    
def load_model(model_path, model_class):
    """Load a saved model"""
    
    device = 'cuda:0'
    model = model_class()
    model.load_state_dict(torch.load(model_path)['model'])
    model.to(device)
    return model

In [13]:
#since length list is all the same is that the problem

In [14]:
fdn = FrameDiffNoise()
B = 16
#single_t dataset
# sd = smallPDBDataset(fdn , meta_data_path = '/mnt/h/datasets/bCov_4H/metadata.csv', 
#                      filter_dict=False, maxlen=1000, input_t=0.05)
#single_t dataset
sd = smallPDBDataset(fdn , meta_data_path = '/mnt/h/datasets/bCov_4H/metadata.csv', 
                     filter_dict=False, maxlen=5000)
ts = TrainSampler(B,sd, sample_mode='single_length')

In [15]:
#ts = TrainSampler(B,sd, sample_mode='length_batch')

In [16]:
gu = GraphUNet(batch_size = B, num_layers_ca = 2).to('cuda')
opti = torch.optim.Adam(gu.parameters(), lr=0.0005, weight_decay=5e-6) #prev lr=0.0005
gm = Make_KNN_MP_Graphs() #consider precalculating graphs for training

In [17]:
len(sd)

5000

In [18]:
dL = torch.utils.data.DataLoader(sd,
        sampler=ts,
        batch_size=B,
        shuffle=False,
        collate_fn=None)

In [19]:
# for batch_feats in dL:
#     device='cuda'
#     batch_feats= tree.map_structure(
#                     lambda x: x.to(device), batch_feats)
#     noised_dict =   {'CA': batch_feats['CA_noised'] ,
#                      'N_CA': batch_feats['N_CA_noised'].unsqueeze(-2) ,
#                      'C_CA': batch_feats['C_CA_noised'].unsqueeze(-2)  }
#     break
# t,n,p = get_noise_pred_true(batch_feats, noised_dict, gm, gu, scale=10)

In [20]:
view_t = np.array([0.01,0.05,0.1,0.2,0.3,0.5,0.8,1.0])
view_t = view_t[None,...].repeat(int(np.ceil(B/len(view_t))),axis=0).flatten()[:B]
visualize_model(sd, view_t, epoch=1, numOut=1, outdir='output/')

/home/nwoodall/miniconda3/envs/se33/lib/python3.9/site-packages/dgl/backend/pytorch/tensor.py:449: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


In [21]:
model_path = make_save_folder(name=f'4h_full_t_load_check')
num_epochs = 300
e_start= 0
save_per=10
avg_vloss=0

#visualize_T
#view_t = np.array([0.05]*8)

view_t = np.array([0.01,0.05,0.1,0.2,0.3,0.5,0.8,1.0])
view_t = view_t[None,...].repeat(int(np.ceil(B/len(view_t))),axis=0).flatten()[:B]


for e in range(e_start, e_start+num_epochs):
    
    running_tloss = 0 
    start = time.time()
    for i,batch_feats in enumerate(dL):

        device='cuda'
        batch_feats= tree.map_structure(
                        lambda x: x.to(device), batch_feats)
        noised_dict =   {'CA': batch_feats['CA_noised'] ,
                         'N_CA': batch_feats['N_CA_noised'].unsqueeze(-2) ,
                         'C_CA': batch_feats['C_CA_noised'].unsqueeze(-2)  }

        pred, train_loss = model_step(batch_feats, noised_dict, gm, gu, train=True)
        opti.zero_grad()
        train_loss.backward()
        opti.step()

        running_tloss += train_loss.detach().cpu()
        
    

    #n,p,t = get_noise_pred_true(batch_feats, noised_dict, gm, gu, scale=10)
    end = time.time()
    avg_tloss = running_tloss/(i+1)
    print(f'Average Train Loss Epoch {e}: {avg_tloss};   Epoch time: {end-start:.0f}')
    if e %save_per==save_per-1:
        visualize_model(sd, view_t, epoch=e, numOut=8, outdir=f'{model_path}')
        save_chkpt(f'{model_path}', gu, opti, e, B, avg_vloss, avg_tloss)
        pass


Average Train Loss Epoch 0: 0.1967860460281372;   Epoch time: 407
Average Train Loss Epoch 1: 0.17869804799556732;   Epoch time: 387
Average Train Loss Epoch 2: 0.17455224692821503;   Epoch time: 384
Average Train Loss Epoch 3: 0.17577950656414032;   Epoch time: 384
Average Train Loss Epoch 4: 0.1731521487236023;   Epoch time: 386
Average Train Loss Epoch 5: 0.17853529751300812;   Epoch time: 385
Average Train Loss Epoch 6: 0.17612414062023163;   Epoch time: 384
Average Train Loss Epoch 7: 0.17582055926322937;   Epoch time: 384
Average Train Loss Epoch 8: 0.1724880188703537;   Epoch time: 385
Average Train Loss Epoch 9: 0.17326298356056213;   Epoch time: 378
Average Train Loss Epoch 10: 0.17529898881912231;   Epoch time: 380
Average Train Loss Epoch 11: 0.1757819652557373;   Epoch time: 386
Average Train Loss Epoch 12: 0.17549477517604828;   Epoch time: 383
Average Train Loss Epoch 13: 0.17425617575645447;   Epoch time: 382
Average Train Loss Epoch 14: 0.17422497272491455;   Epoch time

Average Train Loss Epoch 121: 0.1363457441329956;   Epoch time: 385
Average Train Loss Epoch 122: 0.13643795251846313;   Epoch time: 384
Average Train Loss Epoch 123: 0.13534756004810333;   Epoch time: 381
Average Train Loss Epoch 124: 0.13565130531787872;   Epoch time: 379
Average Train Loss Epoch 125: 0.1351713240146637;   Epoch time: 386
Average Train Loss Epoch 126: 0.13479821383953094;   Epoch time: 385
Average Train Loss Epoch 127: 0.13536608219146729;   Epoch time: 385
Average Train Loss Epoch 128: 0.13483324646949768;   Epoch time: 385
Average Train Loss Epoch 129: 0.13410915434360504;   Epoch time: 385
Average Train Loss Epoch 130: 0.13392497599124908;   Epoch time: 385
Average Train Loss Epoch 131: 0.1333518624305725;   Epoch time: 384
Average Train Loss Epoch 132: 0.13162334263324738;   Epoch time: 387
Average Train Loss Epoch 133: 0.1324034482240677;   Epoch time: 388
Average Train Loss Epoch 134: 0.13062304258346558;   Epoch time: 386
Average Train Loss Epoch 135: 0.131766

Average Train Loss Epoch 241: 0.12298718839883804;   Epoch time: 379
Average Train Loss Epoch 242: 0.12298154830932617;   Epoch time: 378
Average Train Loss Epoch 243: 0.1242702454328537;   Epoch time: 375
Average Train Loss Epoch 244: 0.12410955876111984;   Epoch time: 375
Average Train Loss Epoch 245: 0.12229809165000916;   Epoch time: 384
Average Train Loss Epoch 246: 0.12395744770765305;   Epoch time: 384
Average Train Loss Epoch 247: 0.12277005612850189;   Epoch time: 384
Average Train Loss Epoch 248: 0.1227627769112587;   Epoch time: 376
Average Train Loss Epoch 249: 0.12224075198173523;   Epoch time: 380
Average Train Loss Epoch 250: 0.12251082807779312;   Epoch time: 381
Average Train Loss Epoch 251: 0.12451999634504318;   Epoch time: 383
Average Train Loss Epoch 252: 0.12378067523241043;   Epoch time: 383
Average Train Loss Epoch 253: 0.12171947956085205;   Epoch time: 385
Average Train Loss Epoch 254: 0.12362827360630035;   Epoch time: 384
Average Train Loss Epoch 255: 0.1229

In [34]:
# for batch_feats in dL:
#     device='cuda'
#     batch_feats= tree.map_structure(
#                     lambda x: x.to(device), batch_feats)
#     noised_dict =   {'CA': batch_feats['CA_noised'] ,
#                      'N_CA': batch_feats['N_CA_noised'].unsqueeze(-2) ,
#                      'C_CA': batch_feats['C_CA_noised'].unsqueeze(-2)  }
#     break
# t,n,p = get_noise_pred_true(batch_feats, noised_dict, gm, gu, scale=10)

In [35]:
dump_tnp(t,n,p,batch_feats['t'], e=0, numOut=1,outdir='output/')

NameError: name 'n' is not defined

In [ ]:
#the conversions don't seem correct

In [ ]:
visualize_model(bb_dict, noised_bb, batched_t, epoch, numOut=1, outdir='output/')

In [ ]:
t=0.05
t_vec = np.ones((B,))*t
print(t_vec)
model_path = make_save_folder(name=f'fdiff_modelStep_doublecheck')
num_epochs = 50
e_start= 0
save_per=10
avg_vloss=0

for e in range(e_start, e_start+num_epochs):
    
    running_tloss = 0 
    start = time.time()
    for i, bb_dict in enumerate(train_dL):
        noised_bb, tv, ss = fdn.forward_fixed_nodes(bb_dict,t_vec=t_vec, useR3=useR3)
        tv = tv.to('cuda')
        ss = ss.to('cuda')
        train_loss = model_step(bb_dict, noised_bb, tv, ss, gm, gu)
        opti.zero_grad()
        train_loss.backward()
        opti.step()

        running_tloss += train_loss.detach().cpu()
    
    end = time.time()
    avg_tloss = running_tloss/(i+1)
    print(f'Average Train Loss Epoch {e}: {avg_tloss};   Epoch time: {end-start:.0f}')

    if e %save_per==save_per-1:
        with torch.no_grad():
            running_vloss = 0
            for i, bb_dictv in enumerate(val_dL):
                noised_bb, tv, ss = fdn.forward_fixed_nodes(bb_dictv,t_vec=None, useR3=useR3) #this does the opposite of traditional, upweighting lower
                tv = tv.to('cuda')
                ss = ss.to('cuda')
                valid_loss = model_step(bb_dictv, noised_bb, tv, ss, gm, gu)
                running_vloss += valid_loss
                
        avg_vloss = running_vloss/(i+1)
        print(f'Average Valid Loss Epoch {e}: {avg_vloss}')
                
        noised_bb, tv, ss = fdn.forward_fixed_nodes(bb_dict, t_vec=vis_t)
        tv = tv.to('cuda')
        visualize_model(bb_dict, noised_bb, tv, e, numOut=8,outdir=f'{model_path}')
        save_chkpt(f'{model_path}', gu, opti, e, B, avg_vloss, avg_tloss)